In [127]:
import pandas as pd
import numpy as np

In [128]:
df = pd.read_csv("single_play_2022.csv")

In [129]:
df.drop(['player_name', 'hc_x', 'hc_x_ft', 'hc_y', 'hc_y_ft', 'des', 'runnerOn1B', 'events','runnerOn2B', 'runnerOn3B', 'game_total_pitches', 'rowId', 'call', 'px', 'pz', 'ax', 'ay', 'player_total_pitches'], axis=1, inplace=True)
df = df.loc[:, ~df.columns.str.contains("pfx")]
df = df.loc[:, ~df.columns.str.contains("calc")]
df = df.loc[:, ~df.columns.str.contains("back")]
df = df.loc[:, ~df.columns.str.contains("Unnamed")]
df = df.loc[:, ~df.columns.str.contains("0")]
df = df.loc[:, ~df.columns.str.contains('sz')]
df = df.loc[:, ~df.columns.str.contains('az')]

## Pitch Velocities

In [130]:
pitches = ['FF', 'SL', 'SI', 'CH', 'CU', 'FC', 'KC', 'FS']

for pitch in pitches:
    df[f'{pitch}_velo'] = df[df['pitch_type'] == pitch].groupby('pitcher')['start_speed'].transform(lambda x:x.mean())

for pitch in pitches:
    df[f'{pitch}_spin'] = df[df['pitch_type'] == pitch].groupby('pitcher')['spin_rate'].transform(lambda x:x.mean())
    
cols = ['FF_velo', 'SL_velo', 'SI_velo', 'CH_velo', 'CU_velo', 'FC_velo', 'KC_velo', 'FS_velo', 'FF_spin', 'SL_spin', 'SI_spin', 'CH_spin', 'CU_spin', 'FC_spin', 'KC_spin', 'FS_spin']

for col in cols:
    df[col] = df.groupby('pitcher')[col].transform(lambda x:x.fillna(x.mean()))

## Averages

In [131]:
cols = ['extension', 'hit_angle', 'xba']

for col in cols:
    df[f'{col}_avg'] = df.groupby('pitcher')[col].transform(lambda x:x.mean())

## Pitch and Zone Frequencies

In [132]:
df['pitch_sum'] = df.groupby('pitcher')['pitcher'].transform('size')

for pitch in pitches:
    df[f'{pitch}_freq'] = df[df['pitch_type']==pitch].groupby(['pitcher', 'pitch_type'])['pitcher'].transform('size')/df['pitch_sum']

zones = [1,2,3,4,5,6,7,8,9,11,12,13,14]

for zone in zones:
    df[f'zone_{str(zone)}_freq'] = df[df['zone'] == zone].groupby(['pitcher', 'zone'])['pitcher'].transform('size')/df['pitch_sum']

freq = ['FF_freq', 'SL_freq', 'SI_freq', 'CH_freq', 'CU_freq', 'FC_freq', 'KC_freq', 'FS_freq', 'zone_1_freq', 'zone_2_freq', 'zone_3_freq', 'zone_4_freq', 'zone_5_freq', 'zone_6_freq', 'zone_7_freq','zone_8_freq', 'zone_9_freq',  'zone_11_freq', 'zone_12_freq', 'zone_13_freq', 'zone_14_freq']

for col in freq:
    df[col] = df.groupby('pitcher')[col].transform(lambda x:x.fillna(x.mean()))

df.drop('pitch_sum', axis=1, inplace=True)

## Zones

In [133]:
df['upper_z'] = df['zone_1_freq'] + df['zone_2_freq'] + df['zone_3_freq'] 
df['middle_z'] = df['zone_4_freq'] + df['zone_5_freq'] + df['zone_6_freq']
df['lower_z'] = df['zone_7_freq'] + df['zone_8_freq'] + df['zone_9_freq']
df['strike_freq'] = df['zone_1_freq'] + df['zone_2_freq'] + df['zone_3_freq'] + df['zone_4_freq'] + df['zone_5_freq'] + df['zone_6_freq'] + df['zone_7_freq'] + df['zone_8_freq'] + df['zone_9_freq']
df['ball_freq'] = df['zone_11_freq'] + df['zone_12_freq'] + df['zone_13_freq'] + df['zone_14_freq']

df = df.loc[:,~df.columns.str.contains("zone_")]

## Dummies

In [134]:
df['flyball'] = np.where(df['hit_angle_avg']>=df['hit_angle_avg'].describe()['75%'], 1, 0)
df['avg_fly_ground'] = np.where((df['hit_angle_avg'] < df['hit_angle_avg'].describe()['75%']) & (df['hit_angle_avg'] > df['hit_angle_avg'].describe()['25%']), 1, 0)
df['groundball'] = np.where(df['hit_angle_avg']<=df['hit_angle_avg'].describe()['25%'], 1, 0)

In [135]:
df['tall'] = np.where(df['extension_avg']>=df['extension_avg'].describe()['75%'], 1, 0)
df['medium'] = np.where((df['extension_avg'] < df['extension_avg'].describe()['75%']) & (df['extension_avg'] > df['extension_avg'].describe()['25%']), 1, 0)
df['short'] = np.where(df['extension_avg']<=df['extension_avg'].describe()['25%'], 1, 0)

In [136]:
df['barrel_finder'] = np.where(df['xba_avg']>=df['xba_avg'].describe()['75%'], 1, 0)
df['normal_barrels'] = np.where((df['xba_avg'] < df['xba_avg'].describe()['75%']) & (df['xba_avg'] > df['xba_avg'].describe()['25%']), 1, 0)
df['barrel_misser'] = np.where(df['xba_avg']<=df['xba_avg'].describe()['25%'], 1, 0)

In [137]:
df['high_cheeser'] = np.where(df['upper_z'] >= df['upper_z'].describe()['75%'], 1, 0)
df['middle_middle'] = np.where(df['middle_z'] >= df['middle_z'].describe()['75%'], 1, 0)
df['go_fishing'] = np.where(df['lower_z'] >= df['lower_z'].describe()['75%'], 1, 0)

df['wild'] = np.where(df['ball_freq'] >= df['ball_freq'].describe()['75%'], 1, 0)
df['strike_thrower'] = np.where(df['strike_freq'] >= df['strike_freq'].describe()['75%'], 1, 0)

In [138]:
for pitch in pitches:
    df[f'fast_{pitch}'] = np.where(df[f'{pitch}_velo'] >= df[f'{pitch}_velo'].describe()['75%'], 1, 0)
    df[f'normal_{pitch}'] =  np.where((df[f'{pitch}_velo'] < df[f'{pitch}_velo'].describe()['75%']) & (df[f'{pitch}_velo'] > df[f'{pitch}_velo'].describe()['25%']), 1, 0)
    df[f'slow_{pitch}'] = np.where(df[f'{pitch}_velo'] <= df[f'{pitch}_velo'].describe()['25%'], 1, 0)
    
    df[f'tight_{pitch}'] = np.where(df[f'{pitch}_spin'] >= df[f'{pitch}_spin'].describe()['75%'], 1, 0)
    df[f'normal_spin_{pitch}'] =  np.where((df[f'{pitch}_spin'] < df[f'{pitch}_spin'].describe()['75%']) & (df[f'{pitch}_velo'] > df[f'{pitch}_velo'].describe()['25%']), 1, 0)
    df[f'cement_mixer_{pitch}'] = np.where(df[f'{pitch}_spin'] <= df[f'{pitch}_spin'].describe()['25%'], 1, 0)

## Hitters

Determine which zones they have highest chase rate and which pitchers would dice them. Strike three pitches by zones, homeruns by zones etc.

In [139]:
df.columns.values

array(['play_id', 'inning', 'ab_number', 'cap_index', 'outs', 'batter',
       'stand', 'batter_name', 'pitcher', 'p_throws', 'pitcher_name',
       'team_batting', 'team_fielding', 'team_batting_id',
       'team_fielding_id', 'result', 'strikes', 'balls', 'pre_strikes',
       'pre_balls', 'call_name', 'pitch_type', 'pitch_name',
       'description', 'balls_and_strikes', 'start_speed', 'end_speed',
       'extension', 'plateTime', 'zone', 'spin_rate', 'is_bip_out',
       'pitch_number', 'player_total_pitches_pitch_types', 'game_pk',
       'hit_speed_round', 'hit_speed', 'hit_distance', 'xba', 'hit_angle',
       'is_barrel', 'FF_velo', 'SL_velo', 'SI_velo', 'CH_velo', 'CU_velo',
       'FC_velo', 'KC_velo', 'FS_velo', 'FF_spin', 'SL_spin', 'SI_spin',
       'CH_spin', 'CU_spin', 'FC_spin', 'KC_spin', 'FS_spin',
       'extension_avg', 'hit_angle_avg', 'xba_avg', 'FF_freq', 'SL_freq',
       'SI_freq', 'CH_freq', 'CU_freq', 'FC_freq', 'KC_freq', 'FS_freq',
       'upper_z', 'middle

In [140]:
splits = ['pitcher',
          'p_throws',
          'flyball',
          'avg_fly_ground',
          'groundball', 'tall', 'medium',
       'short', 'barrel_finder', 'normal_barrels', 'barrel_misser',
       'high_cheeser', 'middle_middle', 'go_fishing', 'wild',
       'strike_thrower', 'fast_FF', 'normal_FF', 'slow_FF', 'tight_FF',
       'normal_spin_FF', 'cement_mixer_FF', 'fast_SL', 'normal_SL',
       'slow_SL', 'tight_SL', 'normal_spin_SL', 'cement_mixer_SL',
       'fast_SI', 'normal_SI', 'slow_SI', 'tight_SI', 'normal_spin_SI',
       'cement_mixer_SI', 'fast_CH', 'normal_CH', 'slow_CH', 'tight_CH',
       'normal_spin_CH', 'cement_mixer_CH', 'fast_CU', 'normal_CU',
       'slow_CU', 'tight_CU', 'normal_spin_CU', 'cement_mixer_CU',
       'fast_FC', 'normal_FC', 'slow_FC', 'tight_FC', 'normal_spin_FC',
       'cement_mixer_FC', 'fast_KC', 'normal_KC', 'slow_KC', 'tight_KC',
       'normal_spin_KC', 'cement_mixer_KC', 'fast_FS', 'normal_FS',
       'slow_FS', 'tight_FS', 'normal_spin_FS', 'cement_mixer_FS']

for split in splits:
    df[f'la_vs_{split}'] = df.groupby(['batter',split])['hit_angle'].transform(lambda x:x.mean())
    df[f'ev_vs_{split}'] = df.groupby(['batter', split])['hit_speed'].transform(lambda x:x.mean())

/data/conda/envs/pk43821/pk43821_env/lib/python3.7/site-packages/ipykernel_launcher.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/data/conda/envs/pk43821/pk43821_env/lib/python3.7/site-packages/ipykernel_launcher.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


In [141]:
df['team_batting'].value_counts()

NYM    6212
NYY    6162
PHI    5971
PIT    5907
ARI    5796
DET    5754
CLE    5696
TOR    5545
MIN    5537
STL    5476
BAL    5445
LAA    5437
COL    5357
MIA    5141
MIL    5139
SD     5060
WSH    5052
LAD    5006
SEA    4953
HOU    4930
CIN    4904
TB     4887
ATL    4866
CHC    4757
SF     4750
BOS    4696
CWS    4644
KC     4541
OAK    4509
TEX    4433
Name: team_batting, dtype: int64

## Daily Portion

In [159]:
pitch_dict = {
    'German Marquez': 'SD',
    'Blake Snell': 'COL',
    'Shohei Ohtani': 'OAK',
    'Kyle Muller': 'LAA',
    'Alek Manoah': 'STL',
    'Miles Mikolas': 'TOR',
    'Logan Webb': 'NYY',
    'Gerrit Cole': 'SFG',
    'Max Fried': 'WAS',
    'Patrick Corbin': 'ATL',
    'Kyle Gibson':'BOS',
    'Corey Kluber': 'BAL',
    'Corbin Burnes':'CHC',
    'Marcus Stroman':'MIL',
    'Eduardo Rodriguez':'TBR',
    'Shane McClanahan':'DET',
    'Aaron Nola':'TEX',
    'Jacob DeGrom':'PHI',
    'Mitch Keller':'CIN',
    'Hunter Greene':'PIT',
    'Pablo Lopez':'KC',
    'Zack Greinke':'MIN',
    'Max Scherzer':'MIA',
    'Sandy Alcantara':'NYM',
    'Dylan Cease':"HOU",
    'Framber Valdez':'CHW',
    'Zac Gallen':'LAD',
    'Julio Urias':'ARI',
    'Shane Bieber':'SEA',
    'Luis Castillo':'CLE'}

def daily_gen(splits, pitch_dict, df):
    daily_df = pd.DataFrame()
    la_split = [f'la_vs_{split}' for split in splits]
    ev_split = [f'ev_vs_{split}' for split in splits]
    columns = la_split + ev_split
    for pitcher in pitch_dict.keys():
        if pitcher not in set(list(df['pitcher_name'])):
            print(pitcher)
        else:
            pitcher_df = df[df['pitcher_name'] == pitcher].drop_duplicates(subset='pitcher_name', keep='last')
            if pitcher_df.shape[0] == 0:
                return pitcher
            df1 = df[df['team_batting'] == pitch_dict[pitcher]][['batter_name'] + columns].drop_duplicates(subset='batter_name', keep='last')
            df1['pitcher_name'] = pitcher
            count = 0
            for split in splits:
                df1[split] = pitcher_df.iloc[0, pitcher_df.columns.get_loc(split)]
            daily_df = pd.concat([daily_df, df1])
    daily_df = daily_df[['batter_name','pitcher_name'] + splits + la_split + ev_split]
    for split in splits:
        daily_df[f'ev_vs_{split}'] = np.where(daily_df[split] == 0, np.NaN, daily_df[f'ev_vs_{split}'])
        daily_df[f'la_vs_{split}'] = np.where(daily_df[split] == 0, np.NaN, daily_df[f'la_vs_{split}'])
    daily_df['avg_ev'] = daily_df[ev_split].mean(axis=1)
    daily_df['avg_la'] = daily_df[la_split].mean(axis=1)
    return daily_df[['batter_name', 'pitcher_name', 'avg_ev', 'avg_la']].dropna(axis=0).sort_values('avg_ev', ascending=False)

df1 = daily_gen(splits, pitch_dict, df)

Jacob DeGrom
